# nn explain
nn has two main parts : data and model components


containers are responsible for model components and parameters/buffers are responsible for model data


containers : Module, Sequential, ModuleList, ModuleDict, ParameterList, ParameterDict for module construction


parameters : parameter(...) for model training

buffers    : parameter(...) for model aux 

In [1]:
import torch
import torch.nn as nn

## 0.parameters and buffers
In one model, 

parameter needs to backward and be updated by optimizer.step


buffer needs to be used in backward but not be updated by optimizer.step


both of these data are responsible for the whole module, thus they would be saved by model.state_dict() in form of OrderDict. Moreover, they would be loaded by model.load_state_dict(...)

nn.Parameter(...) should be used in the __init__ function in order to have init para at the first place. 

In [2]:
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.a = nn.Parameter(torch.randn(4,4))
        self.linear = nn.Linear(4,5)
        self.tensor_test = torch.rand((1,1), requires_grad=True)
        print("Not added in nn.Module parameters : {}".format(self.tensor_test))
model = test()
print(model)
for para in model.parameters():
    print(para)

Not added in nn.Module parameters : tensor([[0.4228]], requires_grad=True)
test(
  (linear): Linear(in_features=4, out_features=5, bias=True)
)
Parameter containing:
tensor([[ 0.7117,  1.8141,  0.3526, -1.7719],
        [-0.3638, -0.7159,  0.6690,  1.6504],
        [ 1.6786, -1.1447,  1.5435, -1.0639],
        [-0.7800, -0.3764, -0.5309,  0.7988]], requires_grad=True)
Parameter containing:
tensor([[ 0.0989, -0.0407, -0.4824, -0.1058],
        [-0.1064,  0.3533, -0.4064, -0.3245],
        [ 0.2952,  0.4070, -0.0780, -0.0521],
        [-0.1896, -0.1403,  0.1634,  0.2207],
        [ 0.0987,  0.2991,  0.4449, -0.1054]], requires_grad=True)
Parameter containing:
tensor([ 0.4262,  0.3874, -0.4072, -0.2242, -0.1540], requires_grad=True)


In [3]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        
        # fist way to have a parameter
        # self.x = nn.Parameter(...) directly add one var into OrderDict
        self.param1 = nn.Parameter(torch.tensor(1.))
        
        # second way to have a parameter
        # x = nn.Parameter(...) and self.register_parameter() in order to add normal parameter into OrderDict 
        param2 = nn.Parameter(torch.tensor(2.))
        self.register_parameter('param2', param2)
        
        # the only way to have buffer
        # self.register_buffer in order to add normal tensor into OrderDict
        buff = torch.tensor(3.)
        self.register_buffer('buffer', buff)
        
    def forward(self, x):
        # ParameterList can act as an iterable, or be indexed using ints
        x = self.param1
        y = self.param2
        z = torch.mm(x,y)
        return z

model = MyModule()
print("=====para=====")
for para in model.parameters():
    print(para)

print("=====buff=====")
for buff in model.buffers():
    print(buff)

print("=====orderlist=====")
print(model.state_dict())

print("=====save&load=====")
# save model and load
PATH = './MyModule_dict'
torch.save(model.state_dict(), PATH)
model2 = MyModule()
model2.load_state_dict(torch.load(PATH))
print(model2.state_dict())

=====para=====
Parameter containing:
tensor(1., requires_grad=True)
Parameter containing:
tensor(2., requires_grad=True)
=====buff=====
tensor(3.)
=====orderlist=====
OrderedDict([('param1', tensor(1.)), ('param2', tensor(2.)), ('buffer', tensor(3.))])
=====save&load=====
OrderedDict([('param1', tensor(1.)), ('param2', tensor(2.)), ('buffer', tensor(3.))])


## 1. containers include Module, Sequential, ModuleList, ModuleDict, ParameterList, ParameterDict

Among them, nn.Module is the father class and the five following classes should be put under nn.Module class.


These containers can be used for adding module components.

In [4]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(10, 10)) for i in range(10)])

    def forward(self, x):
        # ParameterList can act as an iterable, or be indexed using ints
        for i, p in enumerate(self.params):
            x = self.params[i // 2].mm(x) + p.mm(x)
        return x

model = MyModule()
for para in model.parameters():
    print(para)

Parameter containing:
tensor([[-0.1272,  0.0485, -0.2189,  1.1651, -0.3613, -1.0465,  0.2440, -0.3423,
         -0.1392,  1.2922],
        [ 0.7822, -0.4773, -0.0313, -0.0306, -0.6284,  2.0574,  0.4001,  1.1377,
          2.6969, -0.6150],
        [-0.2943, -0.2042, -0.0590, -0.7561, -0.3240, -0.5563,  1.8937, -0.0426,
          0.8770, -0.5655],
        [-1.8932, -0.0217, -0.1494, -0.8232, -0.1276, -1.1098, -2.0809, -0.4547,
         -2.0309, -1.6300],
        [-0.9356, -0.6837, -0.2891,  0.5182, -1.8027, -0.6758, -0.9304,  0.6134,
          0.5430, -0.4894],
        [-0.2890,  0.5644, -1.1721, -0.2471,  0.5923,  0.8998,  0.4963, -0.9848,
          0.2470,  0.5051],
        [ 1.5218,  0.3686,  1.8202, -2.0923, -0.0528,  0.8303,  1.5411, -1.0907,
         -0.0340, -0.7888],
        [ 1.9285,  0.6090,  0.2700, -1.5023,  0.4229,  1.2153,  0.3863, -0.0705,
          0.3639, -0.1447],
        [ 1.2194,  0.1338, -0.3339, -0.4477, -1.6526,  0.2889,  0.1432, -0.9682,
          1.7001, -0.2439

In [5]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.params = nn.ParameterDict({
                'left': nn.Parameter(torch.randn(5, 10)),
                'right': nn.Parameter(torch.randn(5, 10))
        })

    def forward(self, x, choice):
        # torch.mm() a@b
        # torch.mul() a*b
        x = self.params[choice].mm(x)
        return x
model = MyModule()
model(torch.ones((10,10)), 'left')

tensor([[-7.0917, -7.0917, -7.0917, -7.0917, -7.0917, -7.0917, -7.0917, -7.0917,
         -7.0917, -7.0917],
        [ 0.4745,  0.4745,  0.4745,  0.4745,  0.4745,  0.4745,  0.4745,  0.4745,
          0.4745,  0.4745],
        [-0.5410, -0.5410, -0.5410, -0.5410, -0.5410, -0.5410, -0.5410, -0.5410,
         -0.5410, -0.5410],
        [-1.1596, -1.1596, -1.1596, -1.1596, -1.1596, -1.1596, -1.1596, -1.1596,
         -1.1596, -1.1596],
        [-2.8603, -2.8603, -2.8603, -2.8603, -2.8603, -2.8603, -2.8603, -2.8603,
         -2.8603, -2.8603]], grad_fn=<MmBackward>)

In [6]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.choices = nn.ModuleDict({
                'conv': nn.Conv2d(10, 10, 3),
                'pool': nn.MaxPool2d(3)
        })
        self.activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['prelu', nn.PReLU()]
        ])

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x

model = MyModule()
model(torch.ones((10,10,3,3)), 'conv', 'prelu')

tensor([[[[-0.0569]],

         [[ 0.0111]],

         [[ 0.3262]],

         [[ 0.3203]],

         [[ 0.2505]],

         [[-0.1535]],

         [[-0.0760]],

         [[ 0.6562]],

         [[-0.1165]],

         [[-0.3123]]],


        [[[-0.0569]],

         [[ 0.0111]],

         [[ 0.3262]],

         [[ 0.3203]],

         [[ 0.2505]],

         [[-0.1535]],

         [[-0.0760]],

         [[ 0.6562]],

         [[-0.1165]],

         [[-0.3123]]],


        [[[-0.0569]],

         [[ 0.0111]],

         [[ 0.3262]],

         [[ 0.3203]],

         [[ 0.2505]],

         [[-0.1535]],

         [[-0.0760]],

         [[ 0.6562]],

         [[-0.1165]],

         [[-0.3123]]],


        [[[-0.0569]],

         [[ 0.0111]],

         [[ 0.3262]],

         [[ 0.3203]],

         [[ 0.2505]],

         [[-0.1535]],

         [[-0.0760]],

         [[ 0.6562]],

         [[-0.1165]],

         [[-0.3123]]],


        [[[-0.0569]],

         [[ 0.0111]],

         [[ 0.3262]],

   

## 2.difference between nn.Sequential and nn.Modulelist
both of them are subclasses of containers in torch.nn

The sequential class stores sequential list.

In [7]:
class seq_net(nn.Module):
    def __init__(self):
        super(seq_net, self).__init__()
        self.seq = nn.Sequential(
                   nn.Conv2d(1,20,5),
                   nn.ReLU(),
                   nn.Conv2d(20,64,5),
                   nn.ReLU()
                   )
    def forward(self, x):
        return self.seq(x)

model = seq_net()
print(model)

seq_net(
  (seq): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
  )
)


The ModuleList can be used as list, all elements can be used as elements in the list, but the modules in the list are registered automatically to the whole net and the parameters are automatically put on the whole nn.Module model.

In [8]:
class modlist_net(nn.Module):
    def __init__(self):
        super(modlist_net, self).__init__()
        self.modlist = nn.ModuleList([
                       nn.Conv2d(1, 20, 5),
                       nn.ReLU(),
                       nn.Conv2d(20, 64, 5),
                       nn.ReLU()
                       ])

    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x

model = modlist_net()
print(model)

modlist_net(
  (modlist): ModuleList(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
  )
)


Diff 1 : nn.ModuleList has no forward functions but nn.Sequential has default forward functions

Diff 2 :  nn.Sequential can be named using OrderedDict but nn.ModuleList cannot.

In [9]:
from collections import OrderedDict
class seq_net(nn.Module):
    def __init__(self):
        super(seq_net, self).__init__()
        self.seq = nn.Sequential(OrderedDict([
                   ('conv1', nn.Conv2d(1,20,5)),
                   ('relu1', nn.ReLU()),
                   ('conv2', nn.Conv2d(20,64,5)),
                   ('relu2', nn.ReLU())
                   ]))
    def forward(self, x):
        return self.seq(x)

model = seq_net()
print(model)

seq_net(
  (seq): Sequential(
    (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
  )
)


Diff 3 : module in nn.ModuleList has no order, we can put modules in casual order.

Diff 4 : we can use "for" for duplicate modules in nn.ModuleList.

In [10]:
class modlist_net(nn.Module):
    def __init__(self):
        super(modlist_net, self).__init__()
        self.modlist = nn.ModuleList([nn.Linear(10,10) for i in range(10)]

                                    )
    def forward(self, x):
        for m in self.modlist:
            x = m(x)
        return x

model = modlist_net()
print(model)

modlist_net(
  (modlist): ModuleList(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): Linear(in_features=10, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=10, bias=True)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): Linear(in_features=10, out_features=10, bias=True)
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): Linear(in_features=10, out_features=10, bias=True)
  )
)


## 3. Other APIs for nn.Module base class

collect other APIs not mentioned in the above.


train : effect Dropout & BatchNorm layers


eval  : effect Dropout & BatchNorm layers ---> equivalent to self.train(false)


requires_grad_ : change if autograd should record operations on parameters


register_forward_pre_hook : be called every time before forward() is invoked


register_forward_hook : be called every time when forward() is invoked


named_parameters / named_buffers / named_modules / named_children 
parameters / buffers / modules / children

add_module

apply

In [11]:
# when it comes to tensor we use requires_grad_() or requires_grad = False
x = torch.rand((4,4))
x.requires_grad_(False)
x.requires_grad = False
print(x)

# when it comes to nn.Module we use requires_grad_() or requires_grad = False
# this can be used for freezing parameters when fine tuning
# because the grad would not be changed when passing requires_grad_(False) layers
y = nn.Linear(2,2)
y.requires_grad_(False)
y.requires_grad = False
print(y)

class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(10, 10)) for i in range(10)])

    def forward(self, x):
        # ParameterList can act as an iterable, or be indexed using ints
        for i, p in enumerate(self.params):
            x = self.params[i // 2].mm(x) + p.mm(x)
        return x

model = MyModule()
x = model(torch.ones((10,10)))
model.requires_grad_(False)
loss = torch.sum(x)
loss.backward()


tensor([[0.7096, 0.9641, 0.2971, 0.3942],
        [0.1870, 0.7542, 0.4927, 0.4536],
        [0.5679, 0.3549, 0.6074, 0.8638],
        [0.1525, 0.8247, 0.1993, 0.7845]])
Linear(in_features=2, out_features=2, bias=True)
